In [1]:
import pyspark as ps
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf, col, when
import numpy as np
from IPython.display import Image
from IPython.display import display

In [3]:
spark = ps.sql.SparkSession.builder \
                .master("local") \
                .appName("Book recommandation System") \
                .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

ratings_df = spark.read.csv('Goodbooks-10k/ratings.csv', header=True, inferSchema=True)
ratings_df.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [4]:
ratings_df.show(10)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
|      1|   2077|     4|
|      1|   2487|     4|
|      1|   2900|     5|
|      1|   3662|     4|
|      1|   3922|     5|
+-------+-------+------+
only showing top 10 rows



In [5]:
books_df = spark.read.csv('Goodbooks-10k/books.csv', header=True, inferSchema=True)
books_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- best_book_id: integer (nullable = true)
 |-- work_id: integer (nullable = true)
 |-- books_count: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: double (nullable = true)
 |-- authors: string (nullable = true)
 |-- original_publication_year: double (nullable = true)
 |-- original_title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- work_ratings_count: string (nullable = true)
 |-- work_text_reviews_count: string (nullable = true)
 |-- ratings_1: double (nullable = true)
 |-- ratings_2: integer (nullable = true)
 |-- ratings_3: integer (nullable = true)
 |-- ratings_4: integer (nullable = true)
 |-- ratings_5: integer (nullable = true)
 |-- image_url: string (nullable = true)
 |-- small_image_url: string (nullable = true)


In [6]:
books_df.show(5)

+---+-------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id|book_id|best_book_id|work_id|books_count|     isbn|          isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+-------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------

In [7]:
training_df, validation_df = ratings_df.randomSplit([.8, .2])

In [8]:
iterations = 10
regularization_parameter = 0.1 #lambda
rank = 4
errors = []
err = 0

In [9]:
for rank in range(4, 7):
    als = ALS(maxIter=iterations, regParam=regularization_parameter,
              rank=rank, userCol="user_id", itemCol="book_id", ratingCol = "rating")
    model = als.fit(training_df)
    predictions = model.transform(validation_df)
    new_predictions = predictions.filter(col('prediction') != np.nan)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                    predictionCol="prediction")
    rmse = evaluator.evaluate(new_predictions)
    print("Root Mean Square Error value = " + str(rmse))

Root Mean Square Error value = 0.8853709000484015
Root Mean Square Error value = 0.8974480426862737
Root Mean Square Error value = 0.8987069465513021


In [10]:
als = ALS(maxIter=iterations, regParam=regularization_parameter,
          rank=7, userCol="user_id", itemCol="book_id", ratingCol = "rating")
model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root Mean Square Error value = " + str(rmse))

Root Mean Square Error value = 0.9033742307924921


In [11]:
predictions = model.transform(validation_df)
predictions.show( n = 5)

+-------+-------+------+----------+
|book_id|user_id|rating|prediction|
+-------+-------+------+----------+
|    148|  20467|     3| 2.9529386|
|    148|  28767|     3| 2.9450974|
|    148|  22164|     3|  3.221171|
|    148|  24326|     5| 3.4686005|
|    148|  13034|     4| 3.4333458|
+-------+-------+------+----------+
only showing top 5 rows



In [12]:
predictions.join(books_df, "book_id").select("user_id","title","prediction").show(3)

+-------+----------+----------+
|user_id|     title|prediction|
+-------+----------+----------+
|  12466|Lysistrata| 3.8859563|
|  36623|Lysistrata| 4.0373573|
|  21748|Lysistrata|   3.80092|
+-------+----------+----------+
only showing top 3 rows



In [13]:
user_rabie_test = predictions.filter(col("user_id")==35982).join(books_df, "book_id")
.select("user_id","title","image_url", "prediction")
user_rabie_test.show()

SyntaxError: invalid syntax (<ipython-input-13-b2dd9d635534>, line 2)

In [14]:
for_one_user = predictions.filter(col("user_id")==35980).join(books_df, "book_id")
.select("user_id","title","image_url", "prediction")
for_one_user.show()

SyntaxError: invalid syntax (<ipython-input-14-28cd63988bf6>, line 2)

In [15]:
user_rabie_test = predictions.filter(col("user_id")==35981).join(books_df, "book_id").select("user_id","title","image_url", "prediction")
user_rabie_test.show()

+-------+--------------------+--------------------+----------+
|user_id|               title|           image_url|prediction|
+-------+--------------------+--------------------+----------+
|  35982|The Iliad/The Ody...|https://s.gr-asse...| 3.2619503|
|  35982|Veronika Decides ...|https://s.gr-asse...| 3.6908393|
|  35982|    Tropic of Cancer|https://images.gr...| 3.3640935|
+-------+--------------------+--------------------+----------+



In [16]:
for book in user_rabie_test.take(10):
    print(book.title)
    display(Image(url=book.image_url))

The Iliad/The Odyssey


Veronika Decides to Die


Tropic of Cancer


In [17]:
user_rabie_test = predictions.filter(col("user_id")==35981).join(books_df, "book_id").select("user_id","title","image_url", "prediction")
user_rabie_test.show()

+-------+-----+---------+----------+
|user_id|title|image_url|prediction|
+-------+-----+---------+----------+
+-------+-----+---------+----------+

